In [ ]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
from zipfile import ZipFile
import hvplot.pandas
import hvplot.dask
import random
from datetime import date, datetime, timedelta
from tqdm import tqdm

In [ ]:
def load_zipped_csv(file_path):
    zipped_file = [delayed(pd.read_csv)(file_path)]
    return dd.from_delayed(zipped_file)

In [ ]:
def add_total_number_of_days(df):
    df_to_join = df.groupby("user_id").days_since_prior_order.sum().reset_index().rename(columns={"days_since_prior_order":"total_days_on_platform"})
    df_to_join['user_id'] = df_to_join['user_id'].astype(int)
    df = pd.merge(df, df_to_join, on=["user_id"])
    return df

In [ ]:
def add_initial_date(df):
    pass

# Load the data

In [ ]:
df_order_products = load_zipped_csv("../data/order_products__prior.csv.zip")
df_orders = load_zipped_csv("../data/orders.csv.zip")
df_products = load_zipped_csv("../data/products.csv.zip")
df_aisles = load_zipped_csv("../data/aisles.csv.zip")
df_departments = load_zipped_csv("../data/departments.csv.zip")

In [ ]:
df_orders = df_orders[df_orders["eval_set"] == "prior"]
# df_orders = df_orders[df_orders["user_id"].isin([4, 5, 6, 7])]
df_orders['order_id'] = df_orders['order_id'].astype(int)


df_order_products = df_order_products.groupby("order_id")["product_id"].apply(list).compute().to_frame().reset_index()
df_order_products['order_id'] = df_order_products['order_id'].astype(int)

In [ ]:
df = df_orders.merge(df_order_products, on=["order_id"]).compute()

# df = df.join(df_products, on='product_id', rsuffix="_")
# df = df.join(df_aisles, on='aisle_id', rsuffix="_")
# df = df.join(df_departments, on='department_id', rsuffix="_")

In [ ]:
df = add_total_number_of_days(df)

# Create segementation dataset

In [ ]:
df.head(5)

In [ ]:
df[df["user_id"] == 1]

In [ ]:
nb_users = len(df["user_id"].unique())
nb_users

## Complete product dataset

In [ ]:
df_products = df_products.merge(df_aisles.compute(), on=["aisle_id"])
df_products = df_products.merge(df_departments.compute(), on=["department_id"])
df_products = df_products.compute()
df_products

In [ ]:
df_products

In [ ]:
nb_products = len(df_products["product_id"].unique())
nb_products

In [ ]:
sparse_df = []
for i in tqdm(range(nb_users)):
    a = np.empty((nb_products))
    a[:] = np.nan
    empty_array = pd.arrays.SparseArray(a, dtype=float)
    sparse_df.append(empty_array)

In [ ]:
sparse_df

In [ ]:
sparse_df_ = pd.DataFrame(sparse_df)

In [ ]:
sparse_df_